# Semantic Chunking with LangChain

Installing the dependenices we'll be using to explore what Semantic Chunking is - and why it's useful!

In [ ]:
!pip install -qU langchain_experimental langchain_openai langchain_community langchain ragas

In [ ]:
!pip install -qU faiss-cpu tiktoken

We'll be working with "Alice and Wonderland" as our source material - let's load it into memory.

In [ ]:
!wget https://www.gutenberg.org/files/11/11-0.txt -O alice.txt

--2024-04-15 19:01:45--  https://www.gutenberg.org/files/11/11-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 154638 (151K) [text/plain]
Saving to: ‘alice.txt’

alice.txt           100%[===================>] 151.01K   579KB/s    in 0.3s    

2024-04-15 19:01:45 (579 KB/s) - ‘alice.txt’ saved [154638/154638]



In [ ]:
with open("./alice.txt") as f:
  alice_in_wonderland = f.read()

## RecursiveCharacterTextSplitter AKA "Naive Chunking"

Let's look at our documents if we use a traditional non-semantic chunking strategy!

> NOTE: The chunk size chosen here is purely for illustrative purposes.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=0,
    length_function=len,
    is_separator_regex=False
)

In [ ]:
naive_chunks = text_splitter.split_text(alice_in_wonderland)

In [ ]:
for chunk in naive_chunks[10:15]:
  print(chunk + "\n")

dear! Oh dear! I shall be late!” (when she thought it over afterwards,
it occurred to her that she ought to have wondered at this, but at the

time it all seemed quite natural); but when the Rabbit actually _took a
watch out of its waistcoat-pocket_, and looked at it, and then hurried

on, Alice started to her feet, for it flashed across her mind that she
had never before seen a rabbit with either a waistcoat-pocket, or a

watch to take out of it, and burning with curiosity, she ran across the
field after it, and fortunately was just in time to see it pop down a
large rabbit-hole under the hedge.

In another moment down went Alice after it, never once considering how
in the world she was to get out again.



Notice how our chunks wind up split across sentences, and we have similar context split across chunks as well.

There are a number of strategies to counter this problem but we're going to focus on Semantic Chunking.

## Semantic Chunking

Let's start by providing our OpenAI API key.


In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

Enter your OpenAI API Key:··········


Now let's implement the `SemanticChunker`!

We're going to be using the `percentile` threshold as an example - but there's three different strategies you could use (descriptions provided by the [LangChain docs](https://python.langchain.com/docs/modules/data_connection/document_transformers/semantic-chunker) on Semantic Chunking):

- `percentile` (default) - In this method, all differences between sentences are calculated, and then any difference greater than the X percentile is split.

- `standard_deviation` - In this method, any difference greater than X standard deviations is split.

- `interquartile` - In this method, the interquartile distance is used to split chunks.

The basic idea is as follows:

1. Split our document into sentences (based on `.`, `?`, and `!`)
2. Index each sentence based on position
3. Add a `buffer_size` (`int`) of sentences on either side of our selected sentence
4. Calculate distances between groups of sentences
5. Merge groups based on similarity based on the above thresholds



In [ ]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

# Initialize OpenAI Embeddings
openai_embeddings = OpenAIEmbeddings(model="text-embedding-3-large", base_url="https://drchat.xyz")

# Initialize SemanticChunker with OpenAI Embeddings
semantic_chunker = SemanticChunker(openai_embeddings, breakpoint_threshold_type="percentile")


Now we can create our documents.

In [ ]:
semantic_chunks = semantic_chunker.create_documents([alice_in_wonderland])

Let's look at the chunk associated with the above naive chunks.

Notice how much more information is retained and included in each chunk.

Also notice how much larger this chunk is!

In [ ]:
for semantic_chunk in semantic_chunks:
  if "waistcoat-pocket_" in semantic_chunk.page_content:
    print(semantic_chunk.page_content)
    print(len(semantic_chunk.page_content))

﻿﻿*** START OF THE PROJECT GUTENBERG EBOOK ALICE'S ADVENTURES IN
WONDERLAND ***
[Illustration]




Alice’s Adventures in Wonderland

by Lewis Carroll

THE MILLENNIUM FULCRUM EDITION 3.0

Contents

 CHAPTER I. Down the Rabbit-Hole
 CHAPTER II. The Pool of Tears
 CHAPTER III. A Caucus-Race and a Long Tale
 CHAPTER IV. The Rabbit Sends in a Little Bill
 CHAPTER V. Advice from a Caterpillar
 CHAPTER VI. Pig and Pepper
 CHAPTER VII. A Mad Tea-Party
 CHAPTER VIII. The Queen’s Croquet-Ground
 CHAPTER IX. The Mock Turtle’s Story
 CHAPTER X. The Lobster Quadrille
 CHAPTER XI. Who Stole the Tarts? CHAPTER XII. Alice’s Evidence




CHAPTER I. Down the Rabbit-Hole


Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into
the book her sister was reading, but it had no pictures or
conversations in it, “and what is the use of a book,” thought Alice
“without pictures or conversations?”

So she was considering in her own

## Creating a RAG Pipeline Utilizing Semantic Chunking

Let's create a RAG LCEL chain that leverages our created Semantic Chunks.

We'll start by creating our Retriever.

### Retrieval

We're going to use Meta's FAISS-backed vectorstore, and we'll use `text-embedding-3-large` (the same embedding model used to do the semantic chunking)


In [ ]:
from langchain_community.vectorstores import FAISS
openai_embeddings = OpenAIEmbeddings(model="text-embedding-3-large", base_url="https://drchat.xyz")
semantic_chunk_vectorstore = FAISS.from_documents(semantic_chunks, embedding=openai_embeddings)

We will "limit" our semantic retriever to `k = 1` to demonstrate the power of the semantic chunking strategy while maintaining similar token counts between the semantic and naive retrieved context.

In [ ]:
semantic_chunk_retriever = semantic_chunk_vectorstore.as_retriever(search_kwargs={"k" : 1})

In [ ]:
semantic_chunk_retriever.invoke("Who has a pocket watch?")

[Document(page_content='“He won’t stand beating. Now, if you only kept on good terms with him, he’d do almost anything\nyou liked with the clock. For instance, suppose it were nine o’clock in\nthe morning, just time to begin lessons: you’d only have to whisper a\nhint to Time, and round goes the clock in a twinkling! Half-past one,\ntime for dinner!”\n\n(“I only wish it was,” the March Hare said to itself in a whisper.)\n\n“That would be grand, certainly,” said Alice thoughtfully: “but then—I\nshouldn’t be hungry for it, you know.”\n\n“Not at first, perhaps,” said the Hatter: “but you could keep it to\nhalf-past one as long as you liked.”\n\n“Is that the way _you_ manage?” Alice asked. The Hatter shook his head mournfully.')]

### Augmented

We'll create a classic RAG prompt to augment our question with the retrieved context.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

rag_template = """\
Use the following context to answer the user's query. If you cannot answer, please respond with 'I don't know'.

User's Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(rag_template)

### Generation

We'll use the default `ChatOpenAI` model for our generator.

In [ ]:
from langchain_openai import ChatOpenAI

base_model = ChatOpenAI(base_url="https://drchat.xyz")

### LCEL Chain

We'll create our classic LCEL chain here to test the RAG LCEL chain

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

semantic_rag_chain = (
    {"context" : semantic_chunk_retriever, "question" : RunnablePassthrough()}
    | rag_prompt
    | base_model
    | StrOutputParser()
)

Let's test it out!

In [ ]:
semantic_rag_chain.invoke("How does Alice find herself falling down the rabbit hole into Wonderland?")

'Alice finds herself falling down the rabbit hole into Wonderland when she sees a White Rabbit with pink eyes run by her. The Rabbit takes out a watch from its waistcoat-pocket, which surprises Alice. Curiosity gets the better of her, and she chases the Rabbit across the field until she sees it disappear into a large rabbit-hole under a hedge. Without thinking about how she will get out, Alice follows the Rabbit down the rabbit-hole and begins her journey into Wonderland.'

In [ ]:
semantic_rag_chain.invoke("Who is Dinah, and what is their importance to Alice?")

"Dinah is Alice's cat and she is important to Alice because she is an excellent mouse catcher."

These answers seem great!

Let's repeat this process for our naive chunking!

In [ ]:
naive_chunk_vectorstore = FAISS.from_texts(naive_chunks, embedding=openai_embeddings)

Notice that we're going to use `k = 15` here - this is to "make it a fair comparison" between the two strategies.

In [ ]:
naive_chunk_retriever = naive_chunk_vectorstore.as_retriever(search_kwargs={"k" : 15})

In [ ]:
naive_rag_chain = (
    {"context" : naive_chunk_retriever, "question" : RunnablePassthrough()}
    | rag_prompt
    | base_model
    | StrOutputParser()
)

In [ ]:
naive_rag_chain.invoke("How does Alice find herself falling down the rabbit hole into Wonderland?")

'Alice finds herself falling down the rabbit hole into Wonderland when she sees a large rabbit-hole under the hedge and decides to chase after a rabbit that went down it.'

In [ ]:
naive_rag_chain.invoke("Who is Dinah, and what is their importance to Alice?")

"Dinah is Alice's cat and holds importance to Alice as her beloved pet."

These answers are not bad - but they lack a certain depth that the previous answers did.

## Ragas Assessment Comparison

Let's go ahead and leverage a great tool: [Ragas](https://docs.ragas.io/en/stable/getstarted/index.html)

We're going to split our documents utilizing a different chunking strategy to avoid any "cheating" by the naive retriever.

In [ ]:
synthetic_data_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=0,
    length_function=len,
    is_separator_regex=False
)

In [ ]:
synthetic_data_chunks = synthetic_data_splitter.create_documents([alice_in_wonderland])

Then we will create:

- Questions - synthetically generated (`gpt-3.5-turbo`)
- Contexts - created above
- Ground Truths - synthetically generated (`gpt4-1106-preview`)
- Answers - generated from our Semantic RAG Chain

In [ ]:
questions = []
ground_truths_semantic = []
contexts = []
answers = []

question_prompt = """\
You are a teacher preparing a test. Please create a question that can be answered by referencing the following context.

Context:
{context}
"""

question_prompt = ChatPromptTemplate.from_template(question_prompt)

ground_truth_prompt = """\
Use the following context and question to answer this question using *only* the provided context.

Question:
{question}

Context:
{context}
"""

ground_truth_prompt = ChatPromptTemplate.from_template(ground_truth_prompt)

question_chain = question_prompt | ChatOpenAI(model="gpt-3.5-turbo", base_url="https://drchat.xyz") | StrOutputParser()
ground_truth_chain = ground_truth_prompt | ChatOpenAI(model="gpt4-1106-preview", base_url="https://drchat.xyz") | StrOutputParser()

for chunk in synthetic_data_chunks[10:20]:
  questions.append(question_chain.invoke({"context" : chunk.page_content}))
  contexts.append([chunk.page_content])
  ground_truths_semantic.append(ground_truth_chain.invoke({"question" : questions[-1], "context" : contexts[-1]}))
  answers.append(semantic_rag_chain.invoke(questions[-1]))

We'll format those into a dataset!

In [ ]:
from datasets import load_dataset, Dataset

qagc_list = []

for question, answer, context, ground_truth in zip(questions, answers, contexts, ground_truths_semantic):
  qagc_list.append({
      "question" : question,
      "answer" : answer,
      "contexts" : context,
      "ground_truth" : ground_truth
  })

eval_dataset = Dataset.from_list(qagc_list)

In [ ]:
eval_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 10
})

Now we can implement Ragas metrics and evaluate our created dataset.

In [ ]:
from langchain_openai import OpenAI

In [ ]:
gpt3 = OpenAI(model="gpt-3.5-turbo", base_url="https://drchat.xyz")

In [ ]:
from ragas.llms import LangchainLLMWrapper

gpt3_wrapper = LangchainLLMWrapper(gpt3)

In [ ]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

answer_relevancy.llm = gpt3
faithfulness.llm = gpt3
context_recall.llm = gpt3
context_precision.llm = gpt3

In [ ]:
answer_relevancy.embeddings = openai_embeddings
faithfulness.embeddings = openai_embeddings
context_recall.embeddings = openai_embeddings
context_precision.embeddings= openai_embeddings

In [ ]:
print(answer_relevancy)

AnswerRelevancy(embeddings=<ragas.embeddings.base.LangchainEmbeddingsWrapper object at 0x7cc3df0b3670>, llm=OpenAI(client=<openai.resources.completions.Completions object at 0x7cc3ded437c0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x7cc3eb21b250>, model_name='gpt-3.5-turbo', openai_api_key=SecretStr('**********'), openai_api_base='https://drchat.xyz', openai_proxy='', request_timeout=60), name='answer_relevancy', evaluation_mode=<EvaluationMode.qac: 1>, question_generation=Prompt(name='question_generation', instruction='Generate a question for the given answer and Identify if answer is noncommittal. Give noncommittal as 1 if the answer is noncommittal and 0 if the answer is committal. A noncommittal answer is one that is evasive, vague, or ambiguous. For example, "I don\'t know" or "I\'m not sure" are noncommittal answers', output_format_instruction='The output should be a well-formatted JSON instance that conforms to the JSON schema below.\n\nAs an exampl

In [ ]:
from ragas import evaluate

result = evaluate(
    eval_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ], llm = gpt3,
    embeddings=openai_embeddings
)

AttributeError: 'OpenAI' object has no attribute 'set_run_config'

In [ ]:
result